In [25]:
import cv2
import numpy as np
from IPython.display import Video, HTML

In [26]:
video_1 = './video_files/intruder_1.mp4'
saved_video = './saved_video.mp4'
saved_video_ = './big_saved_video.mp4'
capture_video = cv2.VideoCapture(video_1)
width = int(capture_video.get( cv2.CAP_PROP_FRAME_WIDTH))
hieght = int(capture_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = capture_video.get(cv2.CAP_PROP_FPS)

video_out = cv2.VideoWriter(saved_video, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, hieght) )
video_bigout = cv2.VideoWriter(saved_video_, cv2.VideoWriter_fourcc(*'XVID'), fps, (2*width, 2*hieght) )

### Sample Video

In [27]:
Video(video_1)

In [28]:
bg_sub = cv2.createBackgroundSubtractorKNN(200)

In [29]:
def DrawBanner(frame, text, banner_height_percent=0.08, font_scale =0.8, text_color=(255, 255, 0), font_thickness=2):
    banner_height = int(banner_height_percent * frame.shape[0])
    cv2.rectangle(frame, (0,0), (frame.shape[1], banner_height), (0,0,0), thickness=-1)
    left_offset =20
    location =(left_offset, int(10 + (banner_height_percent * frame.shape[0])/2))
    cv2.putText(frame, text, location, cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, font_thickness, cv2.LINE_AA)

In [30]:
kernel = 5 
frame_start = 5
frame_count = 0
max_contours = 3 

while True:
    is_frame, frame = capture_video.read()
    frame_count += 1 
    if is_frame is False:
        break
    else:
        frame_erode_c = frame.copy()
    frame_mask = bg_sub.apply(frame)



    if frame_count > frame_start:
        motion_area = cv2.findNonZero(frame_mask)
        if motion_area is not None: 
            x, y, w, h = cv2.boundingRect(motion_area)
            cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255, 255), 4)
            DrawBanner(frame, 'Intruder Alert')

        frame_mask_erode = cv2.erode(frame_mask, np.ones(kernel, np.uint8))
        motion_area_erode = cv2.findNonZero(frame_mask_erode)
        if motion_area_erode is not None:
            xe, ye, we, he = cv2.boundingRect(motion_area_erode)
            cv2.rectangle(frame_erode_c, (xe, ye), (xe+we, ye+he), (0,255, 255), 4)
            DrawBanner(frame_erode_c, 'Intruder Alert')

        frame_mask_bgr = cv2.cvtColor(frame_mask, cv2.COLOR_GRAY2BGR)
        frame_mask_erode_bgr = cv2.cvtColor(frame_mask_erode, cv2.COLOR_GRAY2BGR)

        contours, hierarchy = cv2.findContours(frame_mask_erode, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
        if len(contours) > 0:
            cv2.drawContours(frame_mask_erode_bgr, contours, -1, (255,0,255), 2 )
            contours_sorted = sorted(contours, key=cv2.contourArea, reverse=True)

            for idx in range(min(max_contours, len(contours_sorted))):
                xc, yc, wc, hc = cv2.boundingRect(contours_sorted[idx])
                if idx == 0:
                    x1 = xc
                    y1 = yc
                    x2 = xc + wc
                    y2 = yc + hc
                else:
                    x1 = min(x1, xc)
                    y1 = min(y1, yc)
                    x2 = max(x2, xc+wc)
                    y2 = max(y2, yc+hc)
            cv2.rectangle(frame_erode_c, (x1, y1), (x2, y2), (255, 0, 0), thickness=3)
            DrawBanner(frame_mask, 'Intruder')

        DrawBanner(frame_mask_bgr, 'Foreground Mask')
        DrawBanner(frame_mask_erode_bgr, 'Foreground Mask (eroded + contours)')

        top_video = np.hstack([frame_mask_bgr, frame])
        bottom_video = np.hstack([frame_mask_erode_bgr, frame_erode_c])
        combined_video = np.vstack([top_video,bottom_video])



        frame_mask_erode
        video_out.write(frame_erode_c)
        video_bigout.write(combined_video)


capture_video.release()
video_bigout.release()
video_out.release()




### Sample Video Of Intruder Alert 

In [31]:
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/OxRixc0h7LI" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>')

In [32]:
kernel = 5 
frame_start = 5

max_contours = 3 

def intruder_detection(input_video_path:str, output_vid_path:str,
                        kernel:int = kernel, buffer :int= frame_start, top_contours:int = max_contours):
    """
    Summary:
        This function performs intruder detection on a given input video and outputs the result to a specified video file.
        
    Args:
        input_video_path (str): Path to the input video file
        output_vid_path (str): Path to the output video file
        kernel (np.ndarray, optional): Erosion kernel used in the processing. Defaults to `kernel`.
        buffer (int, optional): Number of frames to skip before starting the detection. Defaults to `frame_start`.
        top_contours (int, optional): Number of top contours to display in the output video. Defaults to `max_contours`.
        
    Returns:
        None
    
    Example:
        intruder_detection('input.mp4', 'output.avi')
    """
    
    capture_video = cv2.VideoCapture(input_video_path)
    width = int(capture_video.get( cv2.CAP_PROP_FRAME_WIDTH))
    hieght = int(capture_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = capture_video.get(cv2.CAP_PROP_FPS)

    video_out = cv2.VideoWriter(output_vid_path, cv2.VideoWriter_fourcc(*'XVID'), fps, (width, hieght) )

    frame_count = 0

    while True:
        is_frame, frame = capture_video.read()
        frame_count += 1 
        if is_frame is False:
            break
        else:
            frame_erode_c = frame.copy()
        frame_mask = bg_sub.apply(frame)



        if frame_count > buffer:

            frame_mask_erode = cv2.erode(frame_mask, np.ones(kernel, np.uint8))

            frame_mask_erode_bgr = cv2.cvtColor(frame_mask_erode, cv2.COLOR_GRAY2BGR)

            contours, hierarchy = cv2.findContours(frame_mask_erode, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
            if len(contours) > 0:
                cv2.drawContours(frame_mask_erode_bgr, contours, -1, (255,0,255), 2 )
                contours_sorted = sorted(contours, key=cv2.contourArea, reverse=True)

                for idx in range(min(top_contours, len(contours_sorted))):
                    xc, yc, wc, hc = cv2.boundingRect(contours_sorted[idx])
                    if idx == 0:
                        x1 = xc
                        y1 = yc
                        x2 = xc + wc
                        y2 = yc + hc
                    else:
                        x1 = min(x1, xc)
                        y1 = min(y1, yc)
                        x2 = max(x2, xc+wc)
                        y2 = max(y2, yc+hc)
                cv2.rectangle(frame_erode_c, (x1, y1), (x2, y2), (255, 0, 0), thickness=3)

            frame_mask_erode
            video_out.write(frame_erode_c)

    capture_video.release()
    video_out.release()


In [33]:
intruder_detection('./video_files/intruder_2.mp4', 'intruder_detected_or_not.mp4')

In [34]:
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/lWOwZ1iH5OM" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>')

c:\Users\yaqoo\anaconda3\lib\site-packages\IPython\core\display.py:431: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")
